In [ ]:
# Importando as bibliotecas necessárias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [ ]:
# 1. Criando dados sintéticos para o exemplo
# (Em um caso real, você usaria dados reais de imóveis)
np.random.seed(42)

# Criando 1000 exemplos de casas
n_samples = 1000

# Características das casas
tamanho_m2 = np.random.normal(120, 30, n_samples)  # média de 120m², desvio padrão de 30m²
num_quartos = np.random.randint(1, 5, n_samples)
num_banheiros = np.random.randint(1, 4, n_samples)
idade_casa = np.random.randint(0, 40, n_samples)
distancia_centro = np.random.normal(5, 2, n_samples)  # em km

# Criando o preço baseado nas características (simulando uma relação)
preco = (
    tamanho_m2 * 3000 +  # R$ 3000 por m²
    num_quartos * 50000 +  # R$ 50000 por quarto
    num_banheiros * 35000 +  # R$ 35000 por banheiro
    np.maximum(0, 20 - idade_casa) * 5000 +  # Bonus para casas mais novas
    np.maximum(0, 10 - distancia_centro) * 20000 +  # Bonus para localização
    np.random.normal(0, 50000, n_samples)  # Ruído aleatório
)

# Criando o DataFrame
dados = pd.DataFrame({
    'tamanho_m2': tamanho_m2,
    'num_quartos': num_quartos,
    'num_banheiros': num_banheiros,
    'idade_casa': idade_casa,
    'distancia_centro': distancia_centro,
    'preco': preco
})

# Salvando o DataFrame como um arquivo CSV
dados.to_csv('dados_imoveis.csv', index=False)

In [ ]:
# 2. Preparação dos dados
# Separando features (X) e target (y)
X = dados.drop('preco', axis=1)
y = dados['preco']

# Dividindo em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizando os dados (importante para redes neurais)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# 3. Criando o modelo
def criar_modelo_previsao_preco():
    """
    Cria uma rede neural para prever preços de casas.
    
    Arquitetura:
    - Camada de entrada: 5 neurônios (uma para cada característica)
    - Duas camadas ocultas com ReLU
    - Dropout para regularização
    - Camada de saída com 1 neurônio (preço)
    """
    modelo = models.Sequential([
        # Camada de entrada
        layers.Dense(64, activation='relu', input_shape=(5,), name='camada_entrada'),
        
        # Primeira camada oculta
        layers.Dense(32, activation='relu', name='camada_oculta_1'),
        layers.Dropout(0.2, name='dropout_1'),
        
        # Segunda camada oculta
        layers.Dense(16, activation='relu', name='camada_oculta_2'),
        layers.Dropout(0.2, name='dropout_2'),
        
        # Camada de saída
        layers.Dense(1, name='camada_saida')
    ])
    return modelo

In [ ]:
# 4. Compilando e treinando o modelo
modelo = criar_modelo_previsao_preco()

# Compilando o modelo
modelo.compile(optimizer='adam',  # Otimizador adaptativo
              loss='mse',        # Erro quadrático médio
              metrics=['mae'])   # Erro absoluto médio

# Treinando o modelo
historico = modelo.fit(X_train_scaled, y_train,
                      epochs=50,
                      batch_size=32,
                      validation_split=0.2,
                      verbose=1)


In [ ]:
# 5. Funções de análise e visualização
def analisar_predicoes(modelo, X_test_scaled, y_test):
    """
    Analisa as previsões do modelo comparando com os valores reais.
    """
    predicoes = modelo.predict(X_test_scaled)
    
    # Calculando erro percentual
    erro_percentual = np.abs((predicoes.flatten() - y_test) / y_test) * 100
    
    print("\nAnálise das Previsões:")
    print(f"Erro percentual médio: {erro_percentual.mean():.2f}%")
    print(f"Erro percentual mediano: {np.median(erro_percentual):.2f}%")
    
    # Plotando previsões vs valores reais
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, predicoes, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
    plt.xlabel('Preço Real (R$)')
    plt.ylabel('Preço Previsto (R$)')
    plt.title('Previsões vs Valores Reais')
    plt.show()

def fazer_previsao_exemplo(modelo, scaler):
    """
    Faz uma previsão para uma casa exemplo.
    """
    # Casa exemplo
    casa_exemplo = np.array([[
        150,    # tamanho_m2
        3,      # num_quartos
        2,      # num_banheiros
        5,      # idade_casa
        3       # distancia_centro
    ]])
    
    # Normalizando os dados da casa exemplo
    casa_exemplo_scaled = scaler.transform(casa_exemplo)
    
    # Fazendo a previsão
    preco_previsto = modelo.predict(casa_exemplo_scaled)[0][0]
    
    print("\nExemplo de Previsão:")
    print("Características da casa:")
    print(f"- Tamanho: 150m²")
    print(f"- Quartos: 3")
    print(f"- Banheiros: 2")
    print(f"- Idade: 5 anos")
    print(f"- Distância do centro: 3km")
    print(f"\nPreço previsto: R$ {preco_previsto:,.2f}")

In [ ]:
# 6. Executando as análises
print("\nAnálise do Modelo:")
modelo.summary()

# Plotando o histórico de treinamento
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(historico.history['loss'], label='Treino')
plt.plot(historico.history['val_loss'], label='Validação')
plt.title('Loss do Modelo')
plt.xlabel('Época')
plt.ylabel('Erro Quadrático Médio')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(historico.history['mae'], label='Treino')
plt.plot(historico.history['val_mae'], label='Validação')
plt.title('Erro Absoluto Médio')
plt.xlabel('Época')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.show()

# Analisando as previsões
analisar_predicoes(modelo, X_test_scaled, y_test)

# Fazendo uma previsão exemplo
fazer_previsao_exemplo(modelo, scaler)
